In [1]:
#Import Dependencies
import pandas as pd
import json


# Import the world countries internet data from the World Bank

In [3]:
# Import the countries internet data from world bank 
world_data = pd.read_csv('../data/world_bank_internet_data.csv', skipfooter=446, engine='python')
# Replace all ".." with 0
world_data = world_data.replace("..",0)
#Rename the Country and Country Codes column to allow merging later on
world_data.rename(columns={'Country Name': 'Country', 'Country Code': 'Codes'}, inplace=True)
# Remove the brackets and the duplicated value in the Year columns 
world_data = world_data.rename(columns={col: col.split('[')[0] for col in world_data.columns})
# Remove space in Year columns
world_data.columns = world_data.columns.str.strip()
cols = [3,4,5,6,7,8,9,10,11,12]
world_data.drop(world_data.columns[cols],axis=1,inplace=True)
world_data.head()

,Country,Codes,Series Name,2000,2001,2002,2003,2004,2005,2006,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Afghanistan,AFG,Individuals using the Internet (% of population),0,0.004722568,0.004561395,0.087891253,0.10580903,1.224148084,2.107123645,...,5,5.454545455,5.9,7,8.26,11,13.5,16.8,17.6,18.4
1,Afghanistan,AFG,Secure Internet servers,0,0,0,0,0,0,0,...,16,38,46,57,74,426,1577,1965,1047,1362
2,Afghanistan,AFG,Secure Internet servers (per 1 million people),0,0,0,0,0,0,0,...,0.531254164,1.219458267,1.425490598,1.708079913,2.15031248,12.03967054,43.44818099,52.86248045,27.5223881,34.98736306
3,Afghanistan,AFG,Mobile cellular subscriptions,0,0,25000,200000,600000,1200000,2520366,...,13797879,15340115,16807156,18407168,19709038,21602982,23929713,21976355,22580071,22678024
4,Afghanistan,AFG,Mobile cellular subscriptions (per 100 people),0,0,0.1106157,0.84456353,2.426528361,4.677582611,9.534904581,...,45.81362616,49.22797697,52.08357627,55.15951463,57.27106807,61.05463771,65.92913406,59.12084823,59.3560197,58.25581184


In [4]:
#Import the coordinate csv file, rename columns for merging with each other later on
coordinates = pd.read_csv('../data/world_country.csv')
coordinates.rename(columns={'iso_con' : 'Codes', 'lat': 'Latitude', 'lon': 'Longitude'},inplace=True)
coordinates.drop(columns=['Unnamed: 0', 'country'],inplace=True)
coordinates.head()

,Longitude,Latitude,Codes
0,66.238514,33.768006,AFG
1,19.999962,41.000028,ALB
2,2.999983,28.000027,DZA
3,-170.692511,-14.289304,ASM
4,1.573203,42.540717,AND


In [7]:
# Merge the coordinates with tHE world_data DataFrame
final_world = pd.merge(world_data, coordinates, on='Codes', how='left')
# Find out if there are any NaN values for Latitude and Longitude and then drop them 
final_world[final_world.isnull().any(axis=1)] 
final_world.dropna(subset=['Latitude','Longitude'],inplace=True)
final_world.to_json("../data/final_world.json", orient="index", indent=4)

In [8]:
# Turn the Year columns into Rows
data = pd.melt(final_world, id_vars=['Country', 'Codes','Series Name', 'Longitude', 'Latitude'], var_name='Year', value_name='Values')
# Put all the values in the Values column to numeric
# data['Values'] = pd.to_numeric(data['Values'])
data.rename(columns={'Series Name': 'Series Name'},inplace=True)
data['Year']=data['Year'].astype(int)
data['Values']=data['Values'].astype(float)
data.to_json("../data/data.json", orient="index", indent=4)

# Import ITU Regional, Age and Gender Internet Usage Data

### Regions

In [9]:
# Reading the ITU excel file region sheet to clean and turn into json object
temp_df = pd.read_excel('../data/ITU_regional_global_Key_ICT_indicator_aggregates_rev1_Jan_2022.xlsx', header=None, sheet_name = "By BDT region")

# Getting the first table
temp_df.iloc[3:10,0:18]
# Getting the first column names
table_name = temp_df.iloc[2,0]

#dictionary of tables with key as table name and value as dataframe
table_list = {}
# year list
year_list = list(range(2005,2022))
# range start at 4(3), all the way to row 92 with a step of 9
for i in range(4,92,9):
    #table name, 2 rows from where the table start at 0 (1st) column
    table_name = temp_df.iloc[i-2,0]
    # selecting range of rows and range of columns to extract a table into a dataframe 
    region_df = temp_df.iloc[i:i+6,0:18]
    region_df.reset_index(drop=True, inplace=True)
    # Using year_list and table_name as column names
    column_names= [table_name] + year_list
    # Inserting column names
    region_df.columns = column_names
    table_list[table_name]= region_df

### Gender

In [10]:
# Reading the ITU excel file sex and age sheet to clean and turn into gender dataframe for data cleaning
gender_df = pd.read_excel('../data/ITU_regional_global_Key_ICT_indicator_aggregates_rev1_Jan_2022.xlsx', skiprows=(0,1), sheet_name="Internet use by sex & age")

In [11]:
# Get the gender tables from the excel file
gender_df=gender_df.iloc[0:16,0:18] 
#Drop unnecessary columns
cols = [1,2,3,4,5,6,7,8]
gender_df.drop(gender_df.columns[cols],axis=1,inplace=True)
# # Drop unnecessary rows
gender_df = gender_df.drop(0)
gender_df = gender_df.drop(1)
gender_df = gender_df.drop(3)
gender_df = gender_df.drop(4)
gender_df = gender_df.drop(5)
gender_df = gender_df.drop(6)
gender_df = gender_df.drop(7)
gender_df = gender_df.drop(8)
gender_df = gender_df.drop(9)
gender_df.columns=['Percentage of individuals using the Internet, by sex', '% Total 2020','% Female 2020','% Male 2020']
gender_df.reset_index(inplace=True, drop=True)  

### Age

In [14]:
# Reading the ITU excel file sex and age sheet to clean and turn into age dataframe for data cleaning
age_df = pd.read_excel('../data/ITU_regional_global_Key_ICT_indicator_aggregates_rev1_Jan_2022.xlsx', skiprows=range(0,19),skipfooter=10 , sheet_name="Internet use by sex & age")
age_table_name=age_df.iloc[0,0]

In [15]:
# Get the age tables from the excel file
age_df=age_df.iloc[3:17,0:8]
age_df = age_df.drop(4)
age_df = age_df.drop(5)
age_df = age_df.drop(6)
age_df = age_df.drop(7)
age_df = age_df.drop(8)
age_df = age_df.drop(9)
age_df = age_df.drop(10)
cols= [1,2,3,4]
age_df.drop(age_df.columns[cols],axis=1,inplace=True)
age_df.columns=[age_table_name,'% Total 2020','% Youth(15-24) 2020','% Rest of Population 2020']
age_df.reset_index(inplace=True, drop=True)   